It is immediate that every finite language is regular. We will now investigate which of the minimal DFAs computed in previous questions define finite languages. Let $D$ be a DFA with $n$ states. We claim that $\mathcal{L}D$ is infinite if and only if $\mathcal{L}(D)$ contains a word $w$ of length $n \leq |w| \leq 2n-1$.

Assume that such a word exists. Since $|w| \geq n$, the path taken by the automaton to process $w$ involves $|w| + 1 \geq n + 1$ states visited in the sequence. By the pigeonhole principle, since there are only $n$ distinct states in the DFA, at least one state must be visited twice. This implies that the path contains a cycle. We can decompose $w$ into three parts $w = xyz$, where:

*   $x$ is the path from the start to the first occurrence of the repeated state.
*   $y \geq 1$ is the path around the cycle returning to the same state.
*   $z$ is the path from the repeated state to an accepting state.

Since the cycle brings the automaton back to the same state, we can traverse the cycle loop any number of times. Thus, the words $xy^iz$ for all $i \geq 0$ are also accepted by $D$. Since $|y| \geq 1$, each word in the sequence $xy^0z, xy^1z, xy^2z, \dots$ has a strictly increasing length. Therefore, $\mathcal{L}(D)$ contains infinitely many distinct words.

Conversely, assume that $\mathcal{L}(D)$ is infinite. It cannot contain only words of length less than $n$, thus, $\mathcal{L}(D)$ must contain at least one word with length $\geq n$. Let $w$ be the shortest such word. We claim that $|w| \leq 2n - 1$.

Assume otherwise $|w| \geq 2n$. Since $|w| \ge n$, by the pumping lemma, we can write $w = xyz$ where $1 \leq |y| \leq n$, with $xz$ in $\mathcal{L}(D)$. Then
$|xz| = |w| - |y|$ so we have $|w| - n \leq |xz| < |w|$. Using our assumption, we have $|xz| \ge 2n - n = n$. So $xz$ is a word in $\mathcal{L}(D)$ with length $\geq n$ that is strictly shorter than $w$, giving a contradiction. Hence, the shortest word satisfying the length condition must lie in the range $n \leq |w| \le 2n - 1$.

In [38]:
# @title Data

Table_1 = """
2  5  1
10 10 1
1  2  0
8  3  0
9  2  0
9  2  0
1  9  1
4  6  0
3  6  0
9  2  1
"""

Table_2 = """
6 4 7 1
3 6 5 0
2 6 2 0
6 2 2 0
7 7 3 0
3 7 6 0
5 4 6 0
"""

Table_3 = """
9 5 5 1
7 2 6 0
7 4 1 0
1 2 8 0
8 1 6 0
9 9 8 0
9 3 4 0
8 3 5 0
8 3 1 0
"""

Table_4 = """
18  65  1
67  33  1
34  66  1
90  75  1
12  59  1
99  75  1
54  24  1
71  74  1
100 98  1
29  87  1
42  9   1
47  37  1
77  37  1
82  69  1
11  60  1
18  79  1
36  37  1
6   21  1
53  9   1
34  78  1
18  21  1
21  39  1
91  56  1
68  23  1
47  65  1
92  49  1
11  16  1
75  79  1
74  11  1
57  30  1
19  24  1
60  54  1
30  10  1
14  41  1
22  11  1
90  12  1
8   79  1
25  30  1
6   61  1
45  97  1
2   44  1
90  70  1
20  76  1
10  44  1
31  66  1
46  11  1
11  94  1
100 19  1
34  27  1
30  80  1
7   49  1
30  77  1
5   40  1
51  28  1
77  4   1
64  68  1
9   43  1
9   46  1
78  61  1
91  6   1
54  32  1
11  78  1
83  70  1
34  13  1
30  14  1
75  10  1
2   1   1
5   43  1
67  66  1
61  73  1
53  54  1
73  11  1
71  64  1
79  13  1
29  14  1
70  10  1
56  15  1
40  17  1
7   20  1
79  32  1
34  32  1
61  22  1
75  26  1
11  90  1
13  71  1
55  56  1
49  19  1
90  22  1
80  8   1
74  92  1
6   71  1
8   56  1
9   32  1
80  17  1
95  63  1
69  99  1
14  18  1
73  26  1
12  40  1
12  8   1
"""

In [39]:
import numpy as np

def parse_table(table_str):
    '''
    Parses a space-delimited string into a 2D integer list (matrix).
    Handles multi-line strings with potential extra whitespace.
    '''
    # Filter out empty lines to avoid parsing errors
    lines = [line for line in table_str.strip().splitlines() if line.strip()]
    return np.loadtxt(lines, dtype=int)

def get_accessible_states(transition_table):
    '''
    Determines the set of accessible states.
    '''
    # Handle empty case
    if transition_table.size == 0:
        return []

    n_states, cols = transition_table.shape
    k = cols - 1  # Number of alphabet columns

    # Boolean array to keep track of visited states (0-indexed)
    visited = np.zeros(n_states, dtype=bool)

    # Start at state 1 (index 0)
    queue = [0]
    visited[0] = True

    while queue:
        curr_idx = queue.pop(0)

        # Get all target states for the current state across all inputs
        neighbors = transition_table[curr_idx, :k] - 1

        # Iterate through neighbors
        for next_idx in neighbors:
            if not visited[next_idx]:
                visited[next_idx] = True
                queue.append(next_idx)

    # Convert boolean mask to indices, then back to 1-based labels
    accessible_indices = np.where(visited)[0]
    return (accessible_indices + 1)

def remove_inaccessible_states(table):
    '''
    Remove unreachable rows and renumber the remaining states sequentially.
    '''
    # Get the list of accessible state labels (1-based)
    acc_labels = get_accessible_states(table)
    # Convert to 0-based indices
    acc_indices = np.array(acc_labels) - 1

    # Extract only the accessible rows
    sub_table = table[acc_indices].copy()

    # Create a remapping map (Old Label -> New Label)
    # Initialise with 0 or -1
    mapping = np.zeros(len(table) + 1, dtype=int)
    for new_label, old_label in enumerate(acc_labels, start=1):
        mapping[old_label] = new_label

    # Update the transition columns (0 to k-1) with new labels
    k = sub_table.shape[1] - 1

    # Apply mapping to all transition cells
    sub_table[:, :k] = mapping[sub_table[:, :k]]

    return sub_table

def minimise_dfa(table):
    if table.size == 0: return table

    # Remove Inaccessible
    clean_table = remove_inaccessible_states(table)
    n, cols = clean_table.shape
    k = cols - 1

    # Hopcroft's Table Filling
    distinct = np.zeros((n, n), dtype=bool)
    is_accepting = clean_table[:, k] == 1
    distinct = is_accepting[:, None] != is_accepting[None, :]

    changed = True
    while changed:
        changed = False
        for i in range(n):
            for j in range(i + 1, n):
                if not distinct[i, j]:
                    for char_idx in range(k):
                        u = clean_table[i, char_idx] - 1
                        v = clean_table[j, char_idx] - 1
                        if distinct[u, v]:
                            distinct[i, j] = True
                            distinct[j, i] = True
                            changed = True
                            break

    # Rebuild Table
    state_mapping = np.full(n, -1, dtype=int)
    group_count = 0
    for i in range(n):
        if state_mapping[i] == -1:
            state_mapping[i] = group_count
            for j in range(i + 1, n):
                if not distinct[i, j]: state_mapping[j] = group_count
            group_count += 1

    # Preserve start state at index 0
    start_group = state_mapping[0]
    if start_group != 0:
        state_mapping = np.where(state_mapping == 0, -2, state_mapping)
        state_mapping = np.where(state_mapping == start_group, 0, state_mapping)
        state_mapping = np.where(state_mapping == -2, start_group, state_mapping)

    min_table = np.zeros((group_count, cols), dtype=int)
    visited_groups = set()
    for i in range(n):
        g = state_mapping[i]
        if g not in visited_groups:
            visited_groups.add(g)
            min_table[g, k] = clean_table[i, k]
            for c in range(k):
                min_table[g, c] = state_mapping[clean_table[i, c] - 1] + 1

    return min_table

The following program determines if a language is finite and counts its size. Our approach is as follows:
1.  Minimise the DFA to remove unreachable states and merge equivalent states, reducing the graph size.
2.  Determine which states can reach an accepting state. States that cannot reach an accepting state are irrelevant for language finiteness.
3.  Perform a DFS on the subgraph of live states.
    *   If a cycle is found among these states, then the language is infinite.
    *   If no cycle is found, then the structure is a directed acyclic graph, and the language is finite.
4.  If it is finite, then use a recursive approach to count the number of valid paths from the start state to accepting states.

In [40]:
def get_co_accessible_states(table):
    '''
    Identifies states that can reach an accepting state.
    '''
    n, cols = table.shape
    k = cols - 1

    # Build reverse graph: which states point to state X?
    # reverse_adj[target] = [source1, source2, ...]
    reverse_adj = [[] for _ in range(n)]
    for u in range(n):
        for c in range(k):
            v = table[u, c] - 1
            reverse_adj[v].append(u)

    # BFS backwards from accepting states
    accepting_indices = np.where(table[:, k] == 1)[0]
    live = set(accepting_indices)
    queue = list(accepting_indices)

    while queue:
        curr = queue.pop(0)
        for prev in reverse_adj[curr]:
            if prev not in live:
                live.add(prev)
                queue.append(prev)

    return live

def analyse_language(table):
    '''
    Determines if the language is finite and counts its size.
    Returns: (is_finite: bool, size: int or None)
    '''
    # Minimise first
    min_table = minimise_dfa(table)
    n, cols = min_table.shape
    k = cols - 1

    if n == 0:
        return True, 0

    # Identify live states (Co-accessible)
    # If a state cannot reach an accept state, then it cannot contribute to an
    # infinite loop that matters (loops in dead states are irrelevant).
    live_states = get_co_accessible_states(min_table)

    # If start state (0) is not live, then the language is empty.
    if 0 not in live_states:
        return True, 0

    # Detect cycles in the subgraph of live states
    visited = set()
    recursion_stack = set()

    def has_cycle(u):
        visited.add(u)
        recursion_stack.add(u)

        for c in range(k):
            v = min_table[u, c] - 1
            # Only consider transitions to other LIVE states
            if v in live_states:
                if v not in visited:
                    if has_cycle(v):
                        return True
                elif v in recursion_stack:
                    return True

        recursion_stack.remove(u)
        return False

    # Check for cycles reachable from start
    if has_cycle(0):
        return False, None  # Infinite

    # Count words
    memo = {}

    def count_paths(u):
        if u in memo: return memo[u]

        # Base count: 1 if this state accepts, else 0
        total = 1 if min_table[u, k] == 1 else 0

        for c in range(k):
            v = min_table[u, c] - 1
            if v in live_states:
                total += count_paths(v)

        memo[u] = total
        return total

    size = count_paths(0)
    return True, size

In [41]:
tables = [Table_1, Table_2, Table_3]
table_names = ["Table 1", "Table 2", "Table 3"]
matrices = [parse_table(table) for table in tables]
results = [analyse_language(matrix) for matrix in matrices]

print(f"{'Table':<8} | {'Type':<8} | {'Size':<6}")
print("-" * 30)
for name, result in zip(table_names, results):
    is_finite, size = result
    if is_finite:
        print(f"{name:<8} | {'Finite':<8} | {size:<6}")
    else:
        print(f"{name:<8} | {'Infinite':<8} | {'Inf':<6}")

Table    | Type     | Size  
------------------------------
Table 1  | Infinite | Inf   
Table 2  | Finite   | 1     
Table 3  | Infinite | Inf   


Let $f(n, k, s)$ be the number of regular languages of size $s$ definable by a
minimal $(n, k)$-DFA. We can determine the distribution of finite language sizes.

First, let us theoretically answer the case when $s \geq 2^n$. We have already proved that a language $\mathcal{L}(D)$ is infinite if and only if it accepts a word of length $\geq n$. This means every word $w$ in a finite language defined by an $n$-state DFA must satisfy $|w| \leq n-1$. The total number of possible distinct words of length less than $n-1$ with an alphabet of size $k=2$ is
\begin{equation}
    \sum_{i=0}^{n-1} 2^i = 2^n - 1.
\end{equation}
Therefore, it is impossible for an $n$-state DFA to define a finite language with size $s \ge 2^n$. Hence, $f(n, 2, s) = 0$ for all $s \ge 2^n$.

In [57]:
import itertools

def canonicalise(table):
    '''
    Converts table to canonical form (BFS numbering) for set storage.
    '''
    n, cols = table.shape
    k = cols - 1
    mapping = np.full(n, -1, dtype=int)
    mapping[0] = 0
    next_id = 1
    queue = [0]

    while queue:
        u = queue.pop(0)
        for c in range(k):
            v = table[u, c] - 1
            if mapping[v] == -1:
                mapping[v] = next_id
                next_id += 1
                queue.append(v)

    new_table = np.zeros_like(table)
    for u in range(n):
        new_u = mapping[u]
        new_table[new_u, k] = table[u, k]
        for c in range(k):
            old_v = table[u, c] - 1
            new_table[new_u, c] = mapping[old_v] + 1

    return tuple(new_table.flatten())

def get_language_size(table):
    '''
    Returns size of language if finite, else -1.
    '''
    n, cols = table.shape
    k = cols - 1

    # Identify live states (can reach accept)
    # Reverse graph
    rev_adj = [[] for _ in range(n)]
    for u in range(n):
        for c in range(k):
            v = table[u, c] - 1
            rev_adj[v].append(u)

    live = set(np.where(table[:, k] == 1)[0])
    queue = list(live)
    while queue:
        u = queue.pop(0)
        for prev in rev_adj[u]:
            if prev not in live:
                live.add(prev)
                queue.append(prev)

    if 0 not in live: return 0 # Start cannot reach accept

    # Cycle detection on live subgraph
    visited = set()
    rec_stack = set()

    def has_cycle(u):
        visited.add(u)
        rec_stack.add(u)
        for c in range(k):
            v = table[u, c] - 1
            if v in live:
                if v not in visited:
                    if has_cycle(v): return True
                elif v in rec_stack:
                    return True
        rec_stack.remove(u)
        return False

    if has_cycle(0): return -1 # Infinite

    # Count paths
    memo = {}
    def count(u):
        if u in memo: return memo[u]
        total = 1 if table[u, k] == 1 else 0
        for c in range(k):
            v = table[u, c] - 1
            if v in live:
                total += count(v)
        memo[u] = total
        return total

    return count(0)

def get_distinct_languages(n, k=2):
    distinct_languages = set()

    # Generate all transition structures: n states, k inputs -> n^(n*k) combinations
    states = list(range(1, n + 1))

    # Create iterator for one row
    row_options = list(itertools.product(states, repeat=k))

    # Iterate through all possible transition tables
    for transitions_flat in itertools.product(row_options, repeat=n):
        # Construct the transition part of the matrix
        trans_matrix = np.array(transitions_flat, dtype=int)

        # Check accessibility immediately
        # We temporarily append a dummy accept column to satisfy function signature
        dummy_table = np.hstack((trans_matrix, np.zeros((n, 1), dtype=int)))

        accessible = get_accessible_states(dummy_table)

        # If not all n states are accessible, then this cannot be a minimal DFA
        if len(accessible) != n:
            continue

        # Iterate through all 2^n accept/reject configurations
        for accept_config in itertools.product([0, 1], repeat=n):
            accept_col = np.array(accept_config).reshape(n, 1)

            # Full table: [Transitions | Accept]
            full_table = np.hstack((trans_matrix, accept_col))

            # Minimise the DFA
            minimal_table = minimise_dfa(full_table)

            # We only consider the minimal tables with has exactly n states
            if minimal_table.shape[0] == n:
                # Canonicalise to handle isomorphisms
                signature = canonicalise(minimal_table)
                distinct_languages.add(signature)

    return distinct_languages

In [48]:
def compute_f_n_k_s(max_n, k=2):
    for n in range(1, max_n + 1):
        # Set to store unique minimal DFAs (canonical forms)
        unique_minimal_dfas = get_distinct_languages(n, k)

        # Analyse finiteness for all unique DFAs found
        # Initialise counts for s = 0 to 2^n - 1
        max_s = (2**n) - 1
        s_counts = {s: 0 for s in range(max_s + 1)}
        infinite_count = 0

        for sig in unique_minimal_dfas:
            # Reconstruct table from signature
            arr = np.array(sig, dtype=int).reshape(n, k+1)

            size = get_language_size(arr)
            if size == -1:
                infinite_count += 1
            elif size <= max_s:
                s_counts[size] += 1
            else:
                # This should theoretically not happen
                print(f"Found finite size {size} > 2^n - 1")

        # Output Results
        print(f"\nTotal Minimal (n={n}) DFAs: {len(unique_minimal_dfas)}")
        print(f"Finite Languages: {sum(s_counts.values())}")
        print(f"Infinite Languages: {infinite_count}")
        print("f(n, 2, s) counts:")
        for s in range(max_s + 1):
            print(f"  s={s}: {s_counts[s]}")

compute_f_n_k_s(4)


Total Minimal (n=1) DFAs: 2
Finite Languages: 1
Infinite Languages: 1
f(n, 2, s) counts:
  s=0: 1
  s=1: 0

Total Minimal (n=2) DFAs: 24
Finite Languages: 1
Infinite Languages: 23
f(n, 2, s) counts:
  s=0: 0
  s=1: 1
  s=2: 0
  s=3: 0

Total Minimal (n=3) DFAs: 1028
Finite Languages: 6
Infinite Languages: 1022
f(n, 2, s) counts:
  s=0: 0
  s=1: 2
  s=2: 3
  s=3: 1
  s=4: 0
  s=5: 0
  s=6: 0
  s=7: 0

Total Minimal (n=4) DFAs: 56014
Finite Languages: 60
Infinite Languages: 55954
f(n, 2, s) counts:
  s=0: 0
  s=1: 4
  s=2: 16
  s=3: 20
  s=4: 13
  s=5: 5
  s=6: 1
  s=7: 1
  s=8: 0
  s=9: 0
  s=10: 0
  s=11: 0
  s=12: 0
  s=13: 0
  s=14: 0
  s=15: 0


We are looking for minimal DFAs with $n$ states ($k=2$) that define finite languages of size $S_1 = 1$ (languages containing exactly one word) and size $S_2 = 2^{n-1} - 1$, the maximum possible size (all words of length strictly less than $n-1$). We iterate through all minimal $(n, 2)$-DFAs and filter for those defining a finite language. If the size matches $S_1$ or $S_2$, then output the transition table and the list of accepted words.


In [97]:
def get_accepted_words(table):
    """
    Returns a set of accepted words if the language is finite.
    Returns None if the language is infinite.
    """
    n, cols = table.shape
    k = cols - 1

    # Identify live states
    reverse_adj = [[] for _ in range(n)]
    for u in range(n):
        for c in range(k):
            v = table[u, c] - 1
            reverse_adj[v].append(u)

    live_states = set(np.where(table[:, k] == 1)[0])
    queue = list(live_states)

    while queue:
        curr = queue.pop(0)
        for prev in reverse_adj[curr]:
            if prev not in live_states:
                live_states.add(prev)
                queue.append(prev)

    # If the start state is dead, then the language is empty
    if 0 not in live_states:
        return set()

    # Cycle detection, only check cycles consisting of live states
    visited = set()
    recursion_stack = set()

    def has_cycle(u):
        visited.add(u)
        recursion_stack.add(u)
        for c in range(k):
            v = table[u, c] - 1
            if v in live_states:
                if v not in visited:
                    if has_cycle(v): return True
                elif v in recursion_stack:
                    return True
        recursion_stack.remove(u)
        return False

    if has_cycle(0):
        return None # Infinite

    # Collect words by DFS
    words = set()
    # Stack: (state_index, current_string)
    stack = [(0, "")]

    while stack:
        u, s = stack.pop()

        # Check acceptance
        if table[u, k] == 1:
            words.add(s if s else "ε")

        # Traverse
        for c in range(k):
            v = table[u, c] - 1
            if v in live_states:
                next_char = str(c + 1)
                stack.append((v, s + next_char))

    return words

def get_min_and_max_finite_languages(n_max=4):

    for n in range(1, n_max + 1):
        print("=" * 20)
        print(f"Analysis for n = {n}")
        print("=" * 20)
        targets = sorted(list(set([1, (2**(n - 1)) - 1])))

        # Get candidates
        candidates = get_distinct_languages(n, 2)
        found_count = 0

        for table_data in candidates:
            table = np.array(table_data, dtype=int).reshape(n, 3)
            words = get_accepted_words(table)
            if words is not None:
                size = len(words)
                if size in targets:
                    found_count += 1
                    sorted_words = sorted(list(words), key=lambda x: (len(x), x))
                    print(f"\nSize: {size}")
                    print(f"Language: {sorted_words}")
                    print(f"{'S':<3} {'1':<3} {'2':<3} {'Acc':<3}")
                    for i in range(n):
                        row = table[i]
                        print(f"{i+1:<3} {row[0]:<3} {row[1]:<3} {row[2]:<3}")
                    print("")

get_min_and_max_finite_languages()

Analysis for n = 1

Size: 0
Language: []
S   1   2   Acc
1   1   1   0  

Analysis for n = 2

Size: 1
Language: ['ε']
S   1   2   Acc
1   2   2   1  
2   2   2   0  

Analysis for n = 3

Size: 1
Language: ['2']
S   1   2   Acc
1   2   3   0  
2   2   2   0  
3   2   2   1  


Size: 1
Language: ['1']
S   1   2   Acc
1   2   3   0  
2   3   3   1  
3   3   3   0  


Size: 3
Language: ['1', '2', 'ε']
S   1   2   Acc
1   2   2   1  
2   3   3   1  
3   3   3   0  

Analysis for n = 4

Size: 1
Language: ['11']
S   1   2   Acc
1   2   3   0  
2   4   3   0  
3   3   3   0  
4   3   3   1  


Size: 1
Language: ['22']
S   1   2   Acc
1   2   3   0  
2   2   2   0  
3   2   4   0  
4   2   2   1  


Size: 1
Language: ['12']
S   1   2   Acc
1   2   3   0  
2   3   4   0  
3   3   3   0  
4   3   3   1  


Size: 1
Language: ['21']
S   1   2   Acc
1   2   3   0  
2   2   2   0  
3   4   2   0  
4   2   2   1  


Size: 7
Language: ['1', '2', 'ε', '11', '12', '21', '22']
S   1   2   Acc
1   2   2   

For a minimal DFA with $n$ states and $k=2$, the number of size one languages is
\begin{equation}
    f(n, 2, 1) =
    \begin{cases}
        0 & \text{if } n = 1 \\
        2^{n-2} & \text{if } n \geq 2
    \end{cases}
\end{equation}
A regular language of size $1$ contains exactly one word $L = \{w\}$. To recognize $\{w\}$ with a minimal DFA:
1.  There must be a unique path from the start state to an accepting state labeled by $w$;
2.  Any deviation from this path must lead to a non-accepting sink state;
3.  From the accepting state, all transitions must lead to the sink state;
4.  The sink state transitions to itself.

Let $|w| = L$. The path requires $L+1$ distinct states, the start state and one per character. The sink state is distinct from the accepting state. Thus, the total number of states required is $L+2$. For a DFA with $n$ states, we have $n = L + 2$, so the word length must be $L = n - 2$. This is negative for $n = 1$ so is impossible. Otherwise, the number of such languages is exactly the number of distinct words of length $n-2$ over an alphabet of size $2$.

The number of maximum finite language is
\begin{equation}
    f(n, 2, 2^{n-1} - 1) = 1.
\end{equation}

We have established that a DFA with $n$ states defines a finite language if and only if it accepts no words of length $\geq n$. A stricter bound exists for complete DFAs where every state has a transition for every symbol:

1.  If a minimal $n$-state DFA accepts a word $w$ of length $n-1$, then the path for $w$ visits $n$ distinct states, ending at an accepting state $q_{end}$.
2.  Since the DFA is complete, $q_{end}$ must have outgoing transitions. If $q_{end}$ transitions to any state in the path, then a cycle is formed. Since $q_{end}$ is accepting, this makes the language infinite.
3.  Therefore, $q_{end}$ must transition to a sink state outside the path. This would require $n+1$ states which is impossible.

Thus, the maximum length of a word in a finite language is $n-2$. The largest possible finite language consists of all words of length up to $n-2$. The size of this language is
\begin{equation}
    \sum_{i=0}^{n-2} 2^i = 2^{n-1} - 1,
\end{equation}
of which there is only one such unique language, namely $\mathcal{L} = \{ w \in \Sigma^* \mid |w| \leq n-2 \}$.

Finally, we have the result for $s \geq 2^{n-1}$,
\begin{equation}
    f(n, 2, s) = 0.
\end{equation}
The maximum length of a word in a finite language recognized by a minimal $n$-state DFA is $n-2$. Consequently, the maximum number of words such a language can contain is the total number of words of length no more than $n-2$, which is $2^{n-1} - 1$. Any size $s$ greater than this maximum is impossible for a finite language on $n$ states.